In [1]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os
from constants import *
# from bipwallet import wallet
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("key")
print(mnemonic)
PRIVKEY = 'privkey'

# connect to local ETH/ geth
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)
 
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ../hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    # p_status = p.wait()
    return json.loads(output)

# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {ETH, BTC, BTCTEST}
numderive=3

# Just testing the derive wallets function
print(derive_wallets(mnemonic, BTC, numderive))

wallets = {}
for coin in coins:
    wallets[coin] = derive_wallets(mnemonic, coin, numderive)

# print(wallets)

eth_key = wallets[ETH][0][PRIVKEY]
btc_key = wallets[BTC][0][PRIVKEY]
btctest_key = wallets[BTCTEST][0][PRIVKEY]

# print(f'ETH key {eth_key}')
# print(f'BTC key {btc_key}')
# print(f'BTCTEST key {btctest_key}')

# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, privkey):
    if coin == ETH:
        return Account.privateKeyToAccount(privkey)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(privkey)


eth_account = priv_key_to_account(ETH, eth_key)
btctest_acc = priv_key_to_account(BTCTEST, btctest_key)

print(eth_account.address)
print(btctest_acc.address)

# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        estimate = w3.eth.estimateGas({"from": eth_account.address, "to": recipient, "value": amount})

        return {
            "from": eth_account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": estimate,
            "nonce": w3.eth.getTransactionCount(eth_account.address),
            "chainID": w3.eth.chain_id
        }

    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

# print(create_tx(ETH, eth_account, <receiver address>, 1000))


# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_txn(coin, account, recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_account.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

# print(send_txn(ETH, eth_account, <reciever address>, 1000))


home coral tongue welcome exchange labor border inform cousin sheriff coach pause
[{'path': "m/44'/0'/0'/0/0", 'address': '1PGBJ4TRwZD5YnTWFNSBNgBBvScSVGc9RQ', 'xprv': 'xprvA3u58VKYD7qw2KmWiiXPPpz94EPyxyWzienMhrtugYNBHkGwpA8TnRPJURaka58dW5BPcEuosnT7bYQisb5ocT7dd9EffDUnSvGwJoQ6hkX', 'xpub': 'xpub6GtRXzrS3VQEEoqypk4PkxvscGEUNSEr5shxWFJXEsuAAYc6MhSiLDhnKhLToGSX9B2zKxeYVSUD5b4MZY31Hb4ZLzEJiMa5JmLr8vCK6jp', 'privkey': 'L2YXQAmx4XKV3VZ9qYZX2E1akNwTbFzo1KeGpudhafqRqbEhNTWh', 'pubkey': '02fe1b1f10f52233fb7695f39d62a86af42531a5e89b3e69a36b0cd33f32540248', 'pubkeyhash': 'f431998b04e2b8f149d4b4313e7ed9bd42bf1d58', 'index': 0}, {'path': "m/44'/0'/0'/0/1", 'address': '13TjiN4wGUHVghiEVwcTjA2Nx5hAmpdv2T', 'xprv': 'xprvA3u58VKYD7qw4JoWxqom88NGJvkgS14odT24sFkD5aSPfrZXU9KyNraAHkiX8jNdZZuTP2MtSpM4VD1WGQ448W36hYWq2z2PyuKoAxpKzux', 'xpub': 'xpub6GtRXzrS3VQEGnsz4sLmVGJzrxbAqTnezfwffe9pduyNYetg1geDvete94TrZauJagy2ZBj7ZRqX34Ha6JSpMdp3H1XaKVcVUWFWeNrJD6v', 'privkey': 'KybnbboE2y5jvgq5QS8eGzcGa8xWNkzu6PB4K6MtR

In [2]:
from bit import Key
mykey = Key(btctest_key)
mykey.get_balance('usd')

'0'

In [3]:
w3.eth.getBalance("0x7684672089437E89eA2cE1f19A2530B1B43075d3")

904625697166532776746648320380374280103671755200316906558262375061821325312

In [4]:
create_tx(BTCTEST,btctest_acc,'2NFvhUEXz9srP4NKYaoeffMwUv5LAyeZHbQ', 0.0001)
send_txn(BTCTEST,btctest_acc,'2NFvhUEXz9srP4NKYaoeffMwUv5LAyeZHbQ', 0.0001)

0100000001cb949b87e0bada0ab2bce67cb9002fd2312b7080a6e8b9322d2533f2185d2526010000006b483045022100a642844d51cfc1220703a34980b2273f9290d9936260d2e3df6386a73bfcbe08022030ba8ec2e6bd0a4a41e96d1597015a83db049694fa754953bd2e7b6e16cf887801210245e8f5da1176010c64adc95255f842b59a3e49abbb0bd43b63f39da0e9aa702fffffffff02102700000000000017a914f8c9ee3474ac878bb3e699d45b2def2af8bcde798734040000000000001976a9143c894119c4c0f9a310b80ceaf7d56c35f69d5d9c88ac00000000
